In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from time import sleep
import json
import sys
from urllib.parse import quote as qt
!pip install multiprocess
from multiprocess import Pool, Value, Array
import os
import dill
from ctypes import c_char

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 13.4 MB/s 


In [2]:
def get_single_pageview(single_title, api_fix, headers, params):
  access_para = params['access_para']
  granularity_para = params['granularity_para']
  agent_para = params['agent_para']
  start_date = params['start_date']
  end_date = params['end_date']
  title = single_title.split('__')[0]
  article_para = title
  article_para = article_para.replace(" ","_")
  article_para = qt(article_para)
  counter = int(single_title.split('_')[2])
  api_url = api_fix + access_para + '/'+ agent_para + '/' + article_para + '/' + granularity_para + '/' + start_date + '/' + end_date
  try:
      api_result = requests.get(api_url, headers = headers)
      j = api_result.json()
      df = pd.DataFrame.from_dict(j['items'])[['article','timestamp','views']]
      df_len = len(df)
      df['Year'] = df['timestamp'].apply(lambda x: x[0:4])
      df = df.groupby(['Year','article'])['views'].sum().reset_index()
      e = 'code ' + str(api_result.status_code) + ' has no error'
      if counter % 1000 == 0:
          print (str(counter))
  except :
      e =  str(sys.exc_info()[0])
      df = " "
      print(i + " has an " + e)
      if counter % 1000 == 0:
          print (str(counter))
  finally:
      return df, title, e
        

In [10]:
def get_page_view (article_list, api_fix, header, params):  
  args = [(article, api_fix, header, params) for article in article_list['Page_title'].tolist()]
  if __name__ == '__main__':
      pool = Pool(os.cpu_count()) # to minus one if you are on local machine
      page_views, title, error_msg = zip(*pool.starmap(get_single_pageview, args))
      pool.close()
      pool.join()
  df_error = pd.DataFrame.from_dict({
  'title': title,
  'msg': error_msg
  })
  return page_views, df_error

In [4]:
def get_page_view_from_articles(peopleDirectory, pageViewDirectory, api_fix, header, params):
  articleList = pd.read_csv(peopleDirectory, header=0, engine='c')
  articleList = articleList[['Wiki_ID','Page_title']]
  articleList['Page_title'] = articleList['Page_title'] + '__' + pd.Series(articleList.index).apply(str)
  dill.settings['recurse'] = True
  page_views, df_error = get_page_view(articleList, api_fix, header, params)
  page_views.to_csv(pageViewDirectory, index = False, header = True)

In [6]:
from google.colab import drive # this is only required if the code is run on google colab, remove if run on other enviroment
drive.mount('/content/drive') # this is only required if the code is run on google colab, remove if run on other enviroment

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/Github/epistemic-injustice-on-wikipedia

/content/drive/MyDrive/Github/epistemic-injustice-on-wikipedia


In [8]:
apiFixUrl = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/'
RequestHeaders = {
    'User-Agent': 'My User Agent 1.0',
    'From': 'miao.xi@mail.utoronto.ca' }
RequestParams = {'access_para' : 'all-access', 
  'granularity_para' : 'monthly',
  'agent_para' : 'user',
  'start_date' : '20160101',
  'end_date' : '20211231'}

In [ ]:
get_page_view_from_articles(peopleDirectory='./data/female_wikidata.csv', pageViewDirectory='./data/female_wikipedia_pageview.csv', 
                            api_fix = apiFixUrl, header = RequestHeaders, params = RequestParams)

0
15000
30000
45000
1000
16000
31000
